# 2.2.2 Working with Data

<div style="background-color: #f8f9fa; border: 1px solid #e9ecef; border-radius: 8px; padding: 10px; margin: 10px;">
<strong>📋 Workshop Contents</strong>
<ul style="line-height: 1.2;">
<li><a href="#What-Youll-Learn">What You'll Learn</a></li>
<li><a href="#Prerequisites">Prerequisites</a></li>
<li><a href="#Cost-Overview">Cost Overview</a></li>
<li><a href="#Step-1-Copy-CSVs-to-S3">Step 1: Copy CSVs to S3</a></li>
<li><a href="#Step-2-Using-Query-Editor-to-Create-Tables-and-Import-Small-Amount-of-Data">Step 2: Using Query Editor to Create Tables and Import Data</a></li>
<li><a href="#Step-3-Using-Query-Editor-to-Execute-SQL-Queries">Step 3: Using Query Editor to Execute SQL Queries</a></li>
<li><a href="#Step-4-Typical-Query-Tuning-Steps">Step 4: Typical Query Tuning Steps</a></li>
<li><a href="#Next-Steps">Next Steps</a></li>
</ul>
</div>

Welcome to the second part of connecting to Aurora PostgreSQL! In this notebook, we'll create tables and load data using different methods.

## What You'll Learn
- Use Query Editor in RDS Console
- Create tables using DDL statements
- Load data from S3
- Optimize query performance

## Prerequisites
- Completed [2.2.1 Basic Connectivity](2.2.1_Basic_Connectivity.ipynb)
- S3 bucket for storing data files
- Sample data files ready to upload

## Cost Overview 💰

Use [AWS Pricing Calculator](https://calculator.aws/#/) to estimate the cost for your architecture solution. The following a rough cost estimation for the resources created in this notebook.

| Component | Cost (us-east-1) | Notes |
|-----------|------------------|--------|
| S3 Standard Storage | \$0.023/GB-month | Small CSV files (~1MB total) |
| S3 PUT Requests | \$0.0005/1,000 requests | 8 CSV file uploads |
| Data Transfer | Free | S3 to Aurora in same region |
| Aurora Query Execution | Included by Aurora cost | Refer to [Aurora cost](../2.1_Crearting_Your_First_Aurora_Cluster/2.1.2_create_your_first_aurora_postgresql_part2.ipynb) |

💡 **Cost Optimization Tips:**
- CSV files are small (~1MB total) resulting in minimal S3 storage costs
- Data transfer between S3 and Aurora in the same region is free
- Query execution uses existing Aurora cluster capacity
- Consider S3 Intelligent Tiering for larger datasets
- Use S3 lifecycle policies to transition old data to cheaper storage classes

> **Free Tier Benefits**: New AWS accounts get 5GB of S3 Standard storage free for 12 months, easily covering this workshop's data requirements. Check [here for up-to-date Free Tier information](https://aws.amazon.com/free/).

## Important Notes

> **💡 Important**: [Data API cancels an operation and returns a timeout error if the operation doesn't finish processing within 45 seconds](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/data-api.html#data-api-timeouts).

Let's begin working with our data! 🚀

## Step 1: Copy CSVs to S3

First, download sample data for testing purpose. If the CSV files don't exist in the expected location, the script will automatically download them from the provided URL from [AWS Workshop Studio](https://catalog.us-east-1.prod.workshops.aws/workshops/bbc211ed-aba2-4a6a-a2bf-227fffd3ce99/en-US): `https://ws-assets-prod-iad-r-iad-ed304a55c2ca1aee.s3.us-east-1.amazonaws.com/bbc211ed-aba2-4a6a-a2bf-227fffd3ce99/xanadu-app/sql.zip`

In [ ]:
%%bash
# Download CSV files if they don't exist and upload to S3

# Set your S3 bucket name
S3_BUCKET="your-s3-bucket-name"  # Replace with your bucket name

# Check if S3 bucket exists
if ! aws s3api head-bucket --bucket "$S3_BUCKET" 2>/dev/null; then
    echo "❌ S3 bucket '$S3_BUCKET' does not exist or is not accessible"
    echo "Please create the bucket or update the S3_BUCKET variable with a valid bucket name"
    exit 0
fi

echo "✅ S3 bucket '$S3_BUCKET' exists and is accessible"

# Path where CSV files should be located
CSV_PATH="../../3_Building_Your_First_Serverless_Web_App_with_Aurora/rewards-app-example/lambda/sql"

# Create directory if it doesn't exist
mkdir -p "$CSV_PATH"

# Check if files exist
if [ "$(ls -A $CSV_PATH 2>/dev/null)" ]; then
    echo "✓ CSV files found in $CSV_PATH"
else
    echo "CSV directory is empty, downloading files..."
    
    # URL for SQL files
    SQL_URL="https://ws-assets-prod-iad-r-iad-ed304a55c2ca1aee.s3.us-east-1.amazonaws.com/bbc211ed-aba2-4a6a-a2bf-227fffd3ce99/xanadu-app/sql.zip"
    
    # Download and extract
    wget -q "$SQL_URL" -O /tmp/sql.zip
    unzip -q /tmp/sql.zip -d /tmp
    
    # Copy SQL files to CSV_PATH
    if [ -d "/tmp/sql" ]; then
        cp -r /tmp/sql/* "$CSV_PATH/"
        echo "✓ Copied $(ls /tmp/sql | wc -l) SQL files to $CSV_PATH"
    else
        echo "✗ SQL directory not found in zip"
    fi
    
    # Clean up temporary files
    rm -rf /tmp/sql.zip /tmp/sql
fi

# Create S3 directories
aws s3api put-object --bucket $S3_BUCKET --key sql/ddl/
aws s3api put-object --bucket $S3_BUCKET --key sql/data/

# Upload CSV files to S3
echo "Uploading CSV files to S3..."
aws s3 cp $CSV_PATH/images.csv s3://$S3_BUCKET/sql/data/
aws s3 cp $CSV_PATH/customers.csv s3://$S3_BUCKET/sql/data/
aws s3 cp $CSV_PATH/catalog_items.csv s3://$S3_BUCKET/sql/data/
aws s3 cp $CSV_PATH/catalog_images.csv s3://$S3_BUCKET/sql/data/
aws s3 cp $CSV_PATH/points_balances.csv s3://$S3_BUCKET/sql/data/
aws s3 cp $CSV_PATH/transactions.csv s3://$S3_BUCKET/sql/data/
aws s3 cp $CSV_PATH/order_items.csv s3://$S3_BUCKET/sql/data/
aws s3 cp $CSV_PATH/shopping_cart_items.csv s3://$S3_BUCKET/sql/data/

echo "✅ All files uploaded to S3 successfully!"
echo "S3 Structure:"
aws s3 ls --recursive s3://$S3_BUCKET/sql/

## Step 2: Using Query Editor to Create Tables and Import Small Amount of Data

### Creating Tables and Loading Data

[Amazon Aurora query editor](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/query-editor.html) lets you run SQL statements on your Amazon Aurora cluster through the AWS Management Console. We'll explore using [RDS console](https://console.aws.amazon.com/rds/home) to quickly create a few tables and load small amount of data from S3.

![Creating Tables and Loading Data with Amazon Aurora query editor](../images/2.2-query-editor-run-query.gif)

Follow these steps in the [RDS Console](https://console.aws.amazon.com/rds):

1. Check Data API is enabled (Click **Databases** -> Click **Aurora cluster identifier** -> Scroll down to the bottom -> Find **RDS Data API** column -> Confirm **Status of Data API** is **Enabled**)
2. Open [RDS Query Editor](https://console.aws.amazon.com/rds/home?#query-editor:)
3. Connect to your Amazon Aurora database created by [section 2.1](../2.1_Crearting_Your_First_Aurora_Cluster/README.MD)
    - Database instance or cluster: choose the Amazon Aurora cluster from the drop-down list
    - Database username: Connect with a Secret Manager ARN
    - Secrets manager ARN: enter the Secret Manager ARN associated with your Amazon Aurora cluster (Click **Databases** -> Click **Aurora cluster identifier** -> Click **Configuration** tab -> Find **Authentication** column -> Copy **Master credentials ARN**)
    - Enter the name of the database: e.g. mylab (Click **Databases** -> Click **Aurora instance identifier** -> Click **Configuration** tab -> Find **Configuration** column -> Copy **DB name**)
4. Execute [DDLs to create tables](./sql/xpoints_schema.sql) 
5. Execute the [SQLs to import SQL from S3](./sql/xpoints_load_data.sql). Ensure you modify 'your-s3-bucket-name' and 'your-region' in the script.

> Note: [Data API cancels an operation and returns a timeout error if the operation doesn't finish processing within 45 seconds](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/data-api.html#data-api-timeouts).

## Step 3: Using Query Editor to Execute SQL Queries

1. **Check Row Counts**
```sql
-- Copy this query to RDS Query Editor
SELECT 
    schemaname || '.' || relname AS table_name,
    n_live_tup AS row_count
FROM pg_stat_user_tables
WHERE schemaname = 'xpoints'
ORDER BY table_name;
```

2. **Sample Data Preview**
```sql
-- Customers sample
SELECT * FROM xpoints.customers LIMIT 5;

-- Catalog items sample
SELECT * FROM xpoints.catalog_items LIMIT 5;

-- Transactions sample
SELECT * FROM xpoints.transactions LIMIT 5;
```

## Step 4: Typical Query Tuning Steps

Let's look at the typical steps to optimize OLTP query during development:

**OLTP Query** stands for Real-time transaction processing. 

Common OLTP characteristics:
- Quick response time (< 100ms)
- Single-row or small batch operations
- High concurrency
- Transaction consistency

#### Query Optimization Process:
1. Identify slow queries
2. Analyze execution plan
3. Implement improvements
4. Measure results

First, let's setup connection variables for us to the connect to your Aurora cluster.

In [ ]:
%%bash
# Setup connection variables - detect Aurora clusters with cookbook tag
echo "Setup connection variables..."

# Allow user to specify Aurora cluster name
AURORA_CLUSTER=""  # Set this to your Aurora cluster name where the data is loaded to, e.g. aurora-sv2-xxx-cluster
AWS_REGION=""  # Set your AWS region where your Aurora cluster resides, e.g. us-east-1

# Check if AWS_REGION is available
if [ -z "$AWS_REGION" ]; then
    echo "❌ AWS_REGION is not set"
    echo "Please set the AWS_REGION variable"
    exit 0
fi

if [ -z "$AURORA_CLUSTER" ]; then
    echo "No specific cluster provided, searching for clusters with cookbook tag..."
    # Find Aurora clusters with CreationSource tag and matching names - get most recent
    AURORA_CLUSTER=$(aws rds describe-db-clusters --region $AWS_REGION --query 'DBClusters[?(contains(DBClusterIdentifier, `aurora-sv2`) || DBClusterIdentifier==`aurora-demo`) && Tags[?Key==`CreationSource` && Value==`aws-database-cookbook-v2025.8`]] | sort_by(@, &ClusterCreateTime) | [-1].DBClusterIdentifier' --output text)
    
    if [ -z "$AURORA_CLUSTER" ]; then
        echo "❌ No Aurora cluster found with cookbook tag"
        echo "Create an Aurora cluster following the steps in Section 2 - Creating Your First Aurora Cluster"
        echo "Available clusters:"
        aws rds describe-db-clusters --region $AWS_REGION --query 'DBClusters[*].{Identifier:DBClusterIdentifier,Engine:Engine,Status:Status}' --output table
        exit 0
    fi
else
    echo "Using specified Aurora cluster: $AURORA_CLUSTER"
fi

echo "Found Aurora cluster: $AURORA_CLUSTER"

# Get managed secret from Aurora cluster
SECRET_ARN=$(aws rds describe-db-clusters --region $AWS_REGION --db-cluster-identifier $AURORA_CLUSTER --query 'DBClusters[0].MasterUserSecret.SecretArn' --output text 2>/dev/null)

if [ "$SECRET_ARN" != "None" ] && [ ! -z "$SECRET_ARN" ]; then
    SECRET_NAME=$SECRET_ARN
    echo "Using managed secret: $SECRET_NAME"
else
    # Fallback for CloudFormation stacks - find by ClusterEndpoint containing 'aurora-sv2'
    CF_STACK_NAME=""
    for stack in $(aws cloudformation list-stacks --region $AWS_REGION --stack-status-filter CREATE_COMPLETE UPDATE_COMPLETE --query 'StackSummaries[*].StackName' --output text); do
        CLUSTER_ENDPOINT=$(aws cloudformation describe-stacks --region $AWS_REGION --stack-name $stack --query "Stacks[0].Outputs[?OutputKey=='ClusterEndpoint'].OutputValue" --output text 2>/dev/null)
        if [ ! -z "$CLUSTER_ENDPOINT" ] && [ "$CLUSTER_ENDPOINT" != "None" ] && [[ "$CLUSTER_ENDPOINT" == *"aurora-sv2"* ]]; then
            CF_STACK_NAME=$stack
            SECRET_NAME=$(aws cloudformation describe-stacks --region $AWS_REGION --stack-name $stack --query "Stacks[0].Outputs[?OutputKey=='SecretArn'].OutputValue" --output text)
            AURORA_CLUSTER=$(echo $CLUSTER_ENDPOINT | cut -d'.' -f1)
            echo "Using CloudFormation secret: $SECRET_NAME"
            echo "Found Aurora cluster from CloudFormation: $AURORA_CLUSTER"
            break
        fi
    done
    
    if [ -z "$CF_STACK_NAME" ]; then
        echo "❌ No managed secret found"
        exit 1
    fi
fi

# Get cluster endpoint
CLUSTER_ENDPOINT=$(aws rds describe-db-clusters --region $AWS_REGION --db-cluster-identifier $AURORA_CLUSTER --query 'DBClusters[0].Endpoint' --output text)

# Save connection info (no password stored)
cat > .db_vars << EOF
export AURORA_CLUSTER=$AURORA_CLUSTER
export CLUSTER_ENDPOINT=$CLUSTER_ENDPOINT
export SECRET_NAME=$SECRET_NAME
export AWS_REGION=$AWS_REGION
EOF

echo "✅ Database connection variables set"
echo "Cluster: $AURORA_CLUSTER"
echo "Endpoint: $CLUSTER_ENDPOINT"

Now, let's write a query that handles a customer's loyalty points redemption transaction, where a customer named 'nikki_wolf_15@example.ca' is redeeming 1,000 points to purchase a product. The system first records this redemption in the transaction history and then updates their points balance by subtracting 1,000 points from their account, ensuring both actions happen together to maintain accurate records.

In [ ]:
%%bash
# Before Tuning - Analyze query performance
source .db_vars

echo "Testing query performance before optimization..."

# Get credentials from Secrets Manager
SECRET_VALUE=$(aws secretsmanager get-secret-value --region $AWS_REGION --secret-id $SECRET_NAME --query 'SecretString' --output text)
DB_USERNAME=$(echo $SECRET_VALUE | jq -r '.username')
DB_PASSWORD=$(echo $SECRET_VALUE | jq -r '.password')

# Connect using psql with retrieved credentials
psql "host=$CLUSTER_ENDPOINT port=5432 dbname=mylab user=$DB_USERNAME password=$DB_PASSWORD sslmode=require" << 'EOF'
-- Before Tuning
BEGIN;
EXPLAIN (ANALYZE, VERBOSE, BUFFERS)
WITH inserted_tx AS (
    INSERT INTO xpoints.transactions (customer_id, tx_type, points, tx_description)
    SELECT id, 'REDEEM', 1000, 'Product purchase'
    FROM xpoints.customers 
    WHERE username = 'nikki_wolf_15@example.ca'
    RETURNING customer_id
)
UPDATE xpoints.points_balances pb
SET points_balance = points_balance - 1000
FROM xpoints.customers
WHERE pb.customer_id = customers.id
AND customers.username = 'nikki_wolf_15@example.ca';
ROLLBACK;
EOF

The above query uses [**Common Table Expressions (CTE)**](https://www.postgresql.org/docs/current/queries-with.html) features:

**What are CTEs?** CTEs are temporary named result sets defined with the `WITH` clause that exist only during query execution. They make complex queries more readable and can be referenced multiple times.

1. **INSERT CTE with RETURNING**: `inserted_tx AS (INSERT ... RETURNING customer_id)` inserts a transaction and returns the customer_id
2. **UPDATE with JOIN**: Updates points_balances by joining with customers table using the same username filter
3. **EXPLAIN ANALYZE**: Shows execution plan and actual runtime statistics

This approach still has issues:
- Two separate customer lookups (INSERT uses one, UPDATE uses another)
- Sequential scan of customers table (no index on username)
- The RETURNING clause from INSERT CTE is not used in the UPDATE

> **Issues Identified:**
> 1. Multiple subqueries causing repeated table scans
> 2. Missing index on username
> 3. Separate statements increasing latency

In [ ]:
%%bash
# Create necessary index
source .db_vars

echo "Creating index for optimization..."

# Get credentials from Secrets Manager
SECRET_VALUE=$(aws secretsmanager get-secret-value --secret-id $SECRET_NAME --query 'SecretString' --output text)
DB_USERNAME=$(echo $SECRET_VALUE | jq -r '.username')
DB_PASSWORD=$(echo $SECRET_VALUE | jq -r '.password')

psql "host=$CLUSTER_ENDPOINT port=5432 dbname=mylab user=$DB_USERNAME password=$DB_PASSWORD sslmode=require" \
    -c "CREATE INDEX IF NOT EXISTS idx_customers_username ON xpoints.customers(username);"

In [ ]:
%%bash
# After Tuning - Test optimized query performance
source .db_vars

echo "Testing optimized query performance..."

# Get credentials from Secrets Manager
SECRET_VALUE=$(aws secretsmanager get-secret-value --region $AWS_REGION --secret-id $SECRET_NAME --query 'SecretString' --output text)
DB_USERNAME=$(echo $SECRET_VALUE | jq -r '.username')
DB_PASSWORD=$(echo $SECRET_VALUE | jq -r '.password')

# Connect using psql with retrieved credentials
psql "host=$CLUSTER_ENDPOINT port=5432 dbname=mylab user=$DB_USERNAME password=$DB_PASSWORD sslmode=require" << 'EOF'
-- After Tuning
-- Optimized query
BEGIN;
EXPLAIN (ANALYZE, VERBOSE, BUFFERS)
WITH customer_data AS MATERIALIZED (
    SELECT id FROM xpoints.customers WHERE username = 'nikki_wolf_15@example.ca'
),
inserted_tx AS (
    INSERT INTO xpoints.transactions (customer_id, tx_type, points, tx_description)
    SELECT id, 'REDEEM', 1000, 'Product purchase'
    FROM customer_data
    RETURNING customer_id
)
-- Update points balance
UPDATE xpoints.points_balances pb
SET points_balance = points_balance - 1000
FROM customer_data
WHERE pb.customer_id = customer_data.id;
ROLLBACK;
EOF

The optimized query uses advanced PostgreSQL features for better performance:

1. **MATERIALIZED CTE**: `customer_data AS MATERIALIZED` performs the customer lookup once and reuses the result
2. **INSERT with CTE**: Uses the materialized customer data instead of a subquery
3. **UPDATE with JOIN**: Updates points balance by joining with the CTE, avoiding another subquery
4. **Index Usage**: The previously created index on username enables fast customer lookup

This approach eliminates duplicate work and reduces the number of table scans from 3 to 1.

> **Improvements Made:**
> 1. Added index on frequently queried column
> 2. Used CTE to avoid repeated lookups
> 3. Combined operations using CTEs
> 4. Added MATERIALIZED hint for CTE reuse

## Next Steps

Now that you've created tables and loaded data, proceed to [Advanced Connection Management](2.2.3_Advanced_Connection_Management.ipynb) to learn how to:
- Implement IAM authentication
- Use AWS Secrets Manager for credentials
- Set up RDS Proxy for connection pooling
- Monitor and optimize database connections

## Additional Resources 📚

### Aurora & PostgreSQL
- [Aurora Overview](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/CHAP_AuroraOverview.html)
- [AWS Database Blog](https://aws.amazon.com/blogs/database/)
- [PostgreSQL Documentation](https://www.postgresql.org/docs/)

### Performance Tuning
- [Aurora Best Practices](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/Aurora.BestPractices.html)
- [PostgreSQL Query Performance Tuning](https://www.postgresql.org/docs/current/performance-tips.html)
- [Performance Insights](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/USER_PerfInsights.html)
- [PostgreSQL Performance Optimization](https://wiki.postgresql.org/wiki/Performance_Optimization)

### Data Loading & S3 Integration
- [Aurora S3 Integration](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/postgresql-s3-import.html)
- [Query Editor Documentation](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/query-editor.html)

### Monitoring Tools
- [pg_stat_statements](https://www.postgresql.org/docs/current/pgstatstatements.html) - Query performance statistics
- [auto_explain](https://www.postgresql.org/docs/current/auto-explain.html) - Automatic plan logging
- [pgBadger](https://github.com/darold/pgbadger) - Log analysis tool